In [ ]:
!wget  https://www.manythings.org/anki/fra-eng.zip

--2024-02-12 05:48:58--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7833145 (7.5M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.47M  49.5MB/s    in 0.2s    

2024-02-12 05:48:59 (49.5 MB/s) - ‘fra-eng.zip’ saved [7833145/7833145]



In [ ]:
!unzip -q fra-eng.zip -d /content/fra-eng

# Important Library's

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LSTM,Dense

# Loading Data

In [ ]:
# reading every single line from text file
with open('/content/fra-eng/fra.txt', 'r', encoding='utf-8') as f:
  lines=f.read().split('\n')

In [ ]:
type(lines),len(lines)

(list, 229804)

In [ ]:
# Vectorize the data
input_texts=[]
target_texts=[]
input_chars=set()
target_chars=set()

for line in lines[:min(60000,len(lines))-1]:
  # it split every word from sentence
  input_text,target_text,_=line.split('\t')

  target_text= '\t'+ target_text + '\n'
  target_texts.append(target_text)
  input_texts.append(input_text)

  # this is used for find out every character used in input text and need output text
  for c in input_text:
    if c not in input_chars:
      input_chars.add(c)

  for c in target_text:
    if c not in target_chars:
      target_chars.add(c)


In [ ]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hello.',
 'Hello.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Exhale.',
 'Get up.',
 'Get up.',
 'Ge

In [ ]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [ ]:
len(input_texts),len(target_texts)

(74999, 74999)

In [ ]:
input_chars=sorted(list(input_chars))
target_chars=sorted(list(target_chars))

num_e_token=len(input_chars)
num_d_token=len(target_chars)

max_e_seq_len=max([len(txt) for txt in input_texts])
max_d_seq_len=max([len(txt) for txt in target_texts])

In [ ]:
num_e_token,num_d_token,max_e_seq_len,max_d_seq_len

(80, 105, 24, 74)

In [ ]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_chars)]
)
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 '?': 22,
 'A': 23,
 'B': 24,
 'C': 25,
 'D': 26,
 'E': 27,
 'F': 28,
 'G': 29,
 'H': 30,
 'I': 31,
 'J': 32,
 'K': 33,
 'L': 34,
 'M': 35,
 'N': 36,
 'O': 37,
 'P': 38,
 'Q': 39,
 'R': 40,
 'S': 41,
 'T': 42,
 'U': 43,
 'V': 44,
 'W': 45,
 'X': 46,
 'Y': 47,
 'Z': 48,
 'a': 49,
 'b': 50,
 'c': 51,
 'd': 52,
 'e': 53,
 'f': 54,
 'g': 55,
 'h': 56,
 'i': 57,
 'j': 58,
 'k': 59,
 'l': 60,
 'm': 61,
 'n': 62,
 'o': 63,
 'p': 64,
 'q': 65,
 'r': 66,
 's': 67,
 't': 68,
 'u': 69,
 'v': 70,
 'w': 71,
 'x': 72,
 'y': 73,
 'z': 74,
 'é': 75,
 'ï': 76,
 '’': 77,
 '€': 78}

In [ ]:
target_token_index=dict(
    [(char,i) for i,char in enumerate(target_chars)]
)
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'a': 51,
 'b': 52,
 'c': 53,
 'd': 54,
 'e': 55,
 'f': 56,
 'g': 57,
 'h': 58,
 'i': 59,
 'j': 60,
 'k': 61,
 'l': 62,
 'm': 63,
 'n': 64,
 'o': 65,
 'p': 66,
 'q': 67,
 'r': 68,
 's': 69,
 't': 70,
 'u': 71,
 'v': 72,
 'w': 73,
 'x': 74,
 'y': 75,
 'z': 76,
 '\xa0': 77,
 '«': 78,
 '»': 79,
 'À': 80,
 'Ç': 81,
 'É': 82,
 'Ê': 83,
 'Ô': 84,
 'à': 85,
 'â': 86,
 'ç': 87,
 'è': 88,
 'é': 89,
 'ê': 90,
 'ë': 91,
 'î': 92,
 'ï': 93,
 'ô': 94,
 'ù': 95,
 'û': 96,
 'œ': 97,
 '\u2009': 98,
 '‘': 99,


# One hot encoding

In [ ]:
e_input_data=np.zeros(
    (len(input_texts), max_e_seq_len, num_e_token),dtype='float32'
)

d_input_data=np.zeros(
    (len(target_texts), max_d_seq_len, num_d_token),dtype='float32'
)

d_target_data=np.zeros(
    (len(target_texts), max_d_seq_len, num_d_token),dtype='float32'
)

In [ ]:
e_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        e_input_data[i, t, input_token_index[char]] = 1.
    e_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        d_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            d_target_data[i, t - 1, target_token_index[char]] = 1.
    d_input_data[i, t + 1:, target_token_index[' ']] = 1.
    d_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
e_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

# Model Creation

In [ ]:
encoder_inputs=Input(shape=(None,num_e_token))
encoder=LSTM(256,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [ ]:
decoder_inputs=Input(shape=(None, num_d_token))
decoder_lstm=LSTM(256,return_sequences=True, return_state=True)
decoder_outputs,_, _= decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_d_token,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [ ]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model.fit([e_input_data, d_input_data], d_target_data,
          batch_size = 32,
          epochs = 100,
          validation_split = 0.2)

Epoch 1/100
1500/1500 [==============================] - 23s 13ms/step - loss: 0.7552 - acc: 0.7930 - val_loss: 0.7088 - val_acc: 0.7908
Epoch 2/100
1500/1500 [==============================] - 18s 12ms/step - loss: 0.5173 - acc: 0.8469 - val_loss: 0.5968 - val_acc: 0.8223
Epoch 3/100
1500/1500 [==============================] - 18s 12ms/step - loss: 0.4423 - acc: 0.8684 - val_loss: 0.5375 - val_acc: 0.8397
Epoch 4/100
1500/1500 [==============================] - 17s 12ms/step - loss: 0.4006 - acc: 0.8802 - val_loss: 0.4958 - val_acc: 0.8521
Epoch 5/100
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3720 - acc: 0.8886 - val_loss: 0.4707 - val_acc: 0.8591
Epoch 6/100
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3506 - acc: 0.8947 - val_loss: 0.4457 - val_acc: 0.8667
Epoch 7/100
1500/1500 [==============================] - 16s 11ms/step - loss: 0.3335 - acc: 0.8997 - val_loss: 0.4293 - val_acc: 0.8717
Epoch 8/100
1500/1500 [==================

In [ ]:
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_inputs_h=Input(shape=(256,))
decoder_state_inputs_c=Input(shape=(256,))
decoder_state_inputs=[decoder_state_inputs_h,decoder_state_inputs_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_state_inputs)
decoder_state = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model=Model(
    [decoder_inputs]+ decoder_state_inputs,
    [decoder_outputs]+ decoder_state
)

In [ ]:
reverse_input_char_index=dict(
   (i,char) for char,i in input_token_index.items()
)

In [ ]:
reverse_target_char_index=dict(
   (i,char) for char,i in target_token_index.items()
)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq,verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_d_token))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value,verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_d_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_d_token))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(59990,59999):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = e_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: That's to be expected.
Decoded sentence: C'est mon parale.

-
Input sentence: That's totally normal.
Decoded sentence: C'est peu que vous attendiez.

-
Input sentence: That's totally normal.
Decoded sentence: C'est peu que vous attendiez.

-
Input sentence: That's totally normal.
Decoded sentence: C'est peu que vous attendiez.

-
Input sentence: That's totally untrue.
Decoded sentence: C'est peur que Tom te ta retrai.

-
Input sentence: That's true, isn't it?
Decoded sentence: C'est ce que j'ai besoin.

-
Input sentence: That's useful to know.
Decoded sentence: C'est choqué de danser.

-
Input sentence: That's very dangerous.
Decoded sentence: C'est très cuiti.

-
Input sentence: That's very dishonest.
Decoded sentence: C'est très cuiti.

